In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder

In [2]:
data_dir = "F:/TARGET frontier/TXT33/"
data_name = "predict2019"
tail = "fe"
sample_data = pd.read_csv(data_dir+data_name+".csv",encoding="shift-jis")#,na_values=['--']

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
drop_cols = ["コース区分"]
train_data = sample_data.drop(drop_cols,axis=1)

In [5]:
class Feature_engineering():
    def __init__(self,df):
        self.df = df
        self.l = len(df)
        
    def labelencoding(self,col):
        le = LabelEncoder()
        le.fit(self.df[col])
        self.df[col] = le.transform(self.df[col])
        
    def handle_nan(self,col):
        self.df.fillna({col:"--"},inplace=True)
        
    def fill_nan(self,col):
        self.df.fillna({col:"0"},inplace=True)
    
    def handle_mark(self,col):
        self.df[col] = self.df[col].str.replace("△","")
        self.df[col] = self.df[col].str.replace("▲","")
        self.df[col] = self.df[col].str.replace("☆","")
        #["△","▲","☆"]
    
    def handle_class(self,col):
        dic_class = {"500万":"1勝","1000万":"2勝","1600万":"3勝","OP(L)":"ｵｰﾌﾟﾝ","重賞":"Ｇ３"}
        self.df[col].replace(dic_class,inplace=True)
        
    def handle_specific_nan(self,col):
        dic_specific = {"----":np.nan}
        self.df[col].replace(dic_specific,inplace=True)
    
    def drop_row(self,col):
        self.df.dropna(inplace=True,subset = [col])
        self.df.reset_index(drop=True,inplace=True)
    
    def handle_abs(self,col):
        self.df[col] = self.df[col].astype("float64")
        self.df[col] = self.df[col].where(self.df[col]>0 , 0) #0より小さいなら0を入れてやる
    
    def set_validation(self,y_col,ozz_col):
        self.df[ozz_col] = self.df[ozz_col].astype("float64")
        self.l = len(self.df)
        res = np.array([0.0]*self.l)
        
        for i in range(self.l):
            res[i] = (1/math.exp(3*self.df[y_col][i]))*(1/math.exp(1/self.df[ozz_col][i]*5)) #2/math.exp(1/i*3) min(50,self.df[ozz_col][i]) #1/math.exp(3*self.df[y_col][i])
        self.df[y_col] = np.round(res,5)
        


In [6]:
fe = Feature_engineering(train_data)

In [7]:
nan_cols = ["父タイプ名","前走場所","前芝・ダ","前クラス名","前走馬場状態","前走脚質","前好走"]
for i in nan_cols:
    print("processing:",i)
    fe.handle_nan(i)

processing: 父タイプ名
processing: 前走場所
processing: 前芝・ダ
processing: 前クラス名
processing: 前走馬場状態
processing: 前走脚質
processing: 前好走


In [8]:
fill_nan_with_0 = ["複勝配当"]
for i in fill_nan_with_0:
    print("processing:",i)
    fe.fill_nan(i)

processing: 複勝配当


In [9]:
mark_cols = ["斤量","前走斤量"]
for i in mark_cols:
    print("processing:",i)
    fe.handle_mark(i)

processing: 斤量
processing: 前走斤量


In [10]:
cat_cols = ["場所","芝・ダ","クラス名","馬場状態","性別","種牡馬","父タイプ名","前走場所",
                   "前芝・ダ","前クラス名","前走馬場状態","前走脚質","前好走"]
for i in cat_cols:
    print("processing:",i)
    fe.labelencoding(i)

processing: 場所
processing: 芝・ダ
processing: クラス名
processing: 馬場状態
processing: 性別
processing: 種牡馬
processing: 父タイプ名
processing: 前走場所
processing: 前芝・ダ
processing: 前クラス名
processing: 前走馬場状態
processing: 前走脚質
processing: 前好走


In [11]:
specific_nan = ["前走着差タイム","着差"]
for i in specific_nan:
    print("processing",i)
    fe.handle_specific_nan(i)

processing 前走着差タイム
processing 着差


In [12]:
drop_na = ["単勝オッズ"]
for i in drop_na:
    print("processing",i)
    fe.drop_row(i)

processing 単勝オッズ


In [13]:
handle_abs = ["前走着差タイム","着差"]
for i in handle_abs:
    print("processing:",i)
    fe.handle_abs(i)

processing: 前走着差タイム
processing: 着差


In [14]:
set_valid = [("着差","単勝オッズ")]
for i,j in set_valid:
    print("processing:",i,j)
    fe.set_validation(i,j)
    

processing: 着差 単勝オッズ


In [15]:
train_data.head()

,場所,芝・ダ,距離,クラス名,馬場状態,騎手コード,枠番,性別,年齢,キャリア,斤量,間隔,休み明け〜戦目,種牡馬,父タイプ名,調教師コード,前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走騎手コード,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走PCI3,前走RPCI,前PCI,前好走,前走平均1Fタイム,前走着差タイム,前走単勝オッズ,着差,レースID(旧),複勝オッズ下限,複勝オッズ上限,複勝配当,単勝オッズ
0,1,1,2500,3,2,1127,6,2,4,10,57,10.0,1.0,425,4,1133,2,2,2500.0,5,3,1127.0,8.0,4,57,38.27,34.2,60.80,59.2,61.9,0,12.43,0.3,20.9,0.41595,0619521206,2.2,3.2,290,5.7
1,1,1,2500,3,2,1077,1,1,4,17,55,6.0,11.0,334,6,1054,13,2,2400.0,5,4,666.0,2.0,2,55,37.93,34.0,62.70,61.4,61.6,0,12.32,0.6,12.9,0.42229,0619521201,2.4,3.5,310,5.8
2,1,1,2500,3,2,1096,7,1,4,11,55,27.0,1.0,270,6,1100,13,1,3000.0,10,3,1143.0,3.0,2,57,NaN,13.9,NaN,NaN,NaN,1,13.89,4.2,62.6,0.69145,0619521207,20.8,30.9,0,72.5
3,1,1,2500,3,2,5495,5,1,5,17,55,12.0,1.0,334,6,1038,2,2,2500.0,4,3,5339.0,3.0,3,55,38.27,33.8,63.53,62.8,63.2,0,12.40,0.0,2.7,0.04505,0619521205,1.3,1.5,0,2.0
4,1,1,2500,3,2,1169,4,2,4,15,56,2.0,2.0,336,6,1064,13,2,2000.0,5,3,1169.0,4.0,4,56,37.11,34.1,58.70,57.1,58.8,0,12.07,0.7,12.1,0.09281,0619521204,2.1,3.0,0,5.7


In [16]:
sample_data.head(10)

,場所,芝・ダ,距離,コース区分,クラス名,馬場状態,騎手コード,枠番,性別,年齢,キャリア,斤量,間隔,休み明け〜戦目,種牡馬,父タイプ名,調教師コード,前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走騎手コード,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走PCI3,前走RPCI,前PCI,前好走,前走平均1Fタイム,前走着差タイム,前走単勝オッズ,着差,レースID(旧),複勝オッズ下限,複勝オッズ上限,複勝配当,単勝オッズ
0,中山,芝,2500,A,2勝,良,1127,6,牡,4,10,57,10.0,1.0,ルーラーシップ,ネイティヴダンサー系,1133,中山,芝,2500.0,2勝,良,1127.0,8.0,後方,57,38.27,34.2,60.80,59.2,61.9,*,12.43,0.3,20.9,-0.0,0619521206,2.2,3.2,290.0,5.7
1,中山,芝,2500,A,2勝,良,1077,1,牝,4,17,55,6.0,11.0,ディープインパクト,ロイヤルチャージャー系,1054,東京,芝,2400.0,2勝,重,666.0,2.0,中団,55,37.93,34.0,62.70,61.4,61.6,*,12.32,0.6,12.9,0.0,0619521201,2.4,3.5,310.0,5.8
2,中山,芝,2500,A,2勝,良,1096,7,牝,4,11,55,27.0,1.0,サムライハート,ロイヤルチャージャー系,1100,東京,ダ,3000.0,未勝利,良,1143.0,3.0,中団,57,NaN,13.9,NaN,NaN,NaN,NaN,13.89,4.2,62.6,0.1,0619521207,20.8,30.9,NaN,72.5
3,中山,芝,2500,A,2勝,良,5495,5,牝,5,17,55,12.0,1.0,ディープインパクト,ロイヤルチャージャー系,1038,中山,芝,2500.0,1勝,良,5339.0,3.0,先行,55,38.27,33.8,63.53,62.8,63.2,*,12.40,-0.1,2.7,0.2,0619521205,1.3,1.5,NaN,2.0
4,中山,芝,2500,A,2勝,良,1169,4,牡,4,15,56☆,2.0,2.0,ディープブリランテ,ロイヤルチャージャー系,1064,東京,芝,2000.0,2勝,良,1169.0,4.0,後方,56☆,37.11,34.1,58.70,57.1,58.8,*,12.07,0.7,12.1,0.5,0619521204,2.1,3.0,NaN,5.7
5,中山,芝,2500,A,2勝,良,1075,2,牝,4,18,55,20.0,1.0,ノヴェリスト,その他のエクリプス系,1076,函館,芝,2600.0,2勝,良,1014.0,7.0,中団,53,37.77,36.4,55.37,54.0,53.8,*,12.48,0.6,4.9,0.5,0619521202,1.8,2.6,NaN,5.1
6,中山,芝,2500,A,2勝,良,1043,3,牡,5,23,57,2.0,2.0,アドマイヤムーン,ネイティヴダンサー系,1035,福島,芝,2600.0,2勝,良,1117.0,5.0,後方,56,37.05,40.1,52.10,51.4,42.4,NaN,12.58,4.6,21.5,1.8,0619521203,6.6,10.0,NaN,35.3
7,中山,芝,1200,A,OP(L),良,1075,7,牝,6,34,58,14.0,1.0,ダイワメジャー,ロイヤルチャージャー系,1008,札幌,芝,1200.0,Ｇ３,稍,5203.0,1.0,逃げ,55,33.20,36.4,47.17,42.2,41.2,*,11.60,0.4,12.6,-0.4,0619521107,1.1,1.2,110.0,2.3
8,中山,芝,1200,A,OP(L),良,1127,8,牝,5,18,54,7.0,2.0,キングカメハメハ,ネイティヴダンサー系,386,京都,芝,1200.0,OP(L),稍,1127.0,7.0,中団,53,34.40,35.3,50.40,46.6,47.5,*,11.62,0.5,9.4,0.4,0619521109,1.2,1.8,130.0,4.1
9,中山,芝,1200,A,OP(L),良,1085,5,牡,6,38,58,5.0,5.0,オレハマッテルゼ,ロイヤルチャージャー系,1030,京都,芝,1400.0,Ｇ２,稍,1144.0,3.0,中団,56,35.48,34.6,55.40,49.2,52.5,*,11.70,0.6,337.7,0.4,0619521105,3.1,5.9,310.0,18.2


In [17]:
train_data.to_csv(data_dir+data_name+tail+".csv",index=False,encoding="shift-jis")

In [18]:
train_data.dtypes

場所             int32
芝・ダ            int32
距離             int64
クラス名           int32
馬場状態           int32
騎手コード          int64
枠番             int64
性別             int32
年齢             int64
キャリア           int64
斤量            object
間隔           float64
休み明け〜戦目      float64
種牡馬            int32
父タイプ名          int32
調教師コード         int64
前走場所           int32
前芝・ダ           int32
前距離          float64
前クラス名          int32
前走馬場状態         int32
前走騎手コード      float64
前走枠番         float64
前走脚質           int32
前走斤量          object
前走Ave-3F     float64
前走上り3F       float64
前走PCI3       float64
前走RPCI       float64
前PCI         float64
前好走            int32
前走平均1Fタイム    float64
前走着差タイム      float64
前走単勝オッズ      float64
着差           float64
レースID(旧)      object
複勝オッズ下限      float64
複勝オッズ上限      float64
複勝配当          object
単勝オッズ        float64
dtype: object

In [19]:
train_data["単勝オッズ"].dropna(inplace=True)

In [20]:
for i,j in enumerate(train_data.columns):
    print(i,j)

0 場所
1 芝・ダ
2 距離
3 クラス名
4 馬場状態
5 騎手コード
6 枠番
7 性別
8 年齢
9 キャリア
10 斤量
11 間隔
12 休み明け〜戦目
13 種牡馬
14 父タイプ名
15 調教師コード
16 前走場所
17 前芝・ダ
18 前距離
19 前クラス名
20 前走馬場状態
21 前走騎手コード
22 前走枠番
23 前走脚質
24 前走斤量
25 前走Ave-3F
26 前走上り3F
27 前走PCI3
28 前走RPCI
29 前PCI
30 前好走
31 前走平均1Fタイム
32 前走着差タイム
33 前走単勝オッズ
34 着差
35 レースID(旧)
36 複勝オッズ下限
37 複勝オッズ上限
38 複勝配当
39 単勝オッズ
